In [ ]:
from catboost import CatBoostClassifier
import pandas as pd

In [ ]:
df = pd.read_csv("DATA/new_test.csv")
df

In [ ]:
model = CatBoostClassifier()
model.load_model("DATA/issue_model")


In [ ]:
test = df[["numeric_sod", "numeric_ans", 'Тип обращения на момент подачи', 'Приоритет',
            'Влияние', 'range_time', 'time_to_solve', 'Система', 'Место',
            'Функциональная группа', 'koef']]
test

In [ ]:
ans = model.predict(test)
ans

In [ ]:
df['Тип обращения итоговый'] = ans
df

In [ ]:
df[df['Статус'] == 0]

In [ ]:
for index, row in df.iterrows():
    if row[8] != row[9] and row[4] == 0:
        df.loc[index, 'Тип обращения итоговый'] = row[8]



In [ ]:
df

In [ ]:
def get_requalification_type(ticketStartType, ticketFinalType):
    if ticketStartType == ticketFinalType:
        return 0
    return 2 if ticketStartType == "Инцидент" else 1
map_type = {0: 'Запрос', 1: "Инцидент"}

In [ ]:
get_requalification_type("Инцидент", "Инцидент")

In [ ]:
submission = df[['id', 'Тип обращения на момент подачи', 'Тип переклассификации', 'Тип обращения итоговый', ]].copy()
submission['Тип обращения на момент подачи'] = submission['Тип обращения на момент подачи'].map(map_type)
submission['Тип обращения итоговый'] = submission['Тип обращения итоговый'].map(map_type)
submission['Тип переклассификации'] = submission.apply(lambda row: get_requalification_type(row['Тип обращения на момент подачи'], row['Тип обращения итоговый']), axis=1)
submission

In [ ]:
submission['Тип переклассификации'].value_counts()

In [ ]:
len(submission)

In [ ]:
submission.drop(['Тип обращения на момент подачи'], axis=1).to_csv("submission.csv", index=False)